<h1> Wykład 4 </h1>

<h1> QGIS: programowanie ciąg dalszy</h1>

<h3> Modyfikacje danych </h3>

Na ostatnich ćwiczeniach ładowaliśmy warstwy, dokonywaliśmy selekcji i uzyskiwaliśmy dostęp do wartości używając klas warstw wektorowych, rastrowych, projektu QGIS oraz dataProvider przekazującego dane. Możliwe jest również wygodne edytowanie parametrów i wartości tych obiektów za pomocą poleceń Pythona.

Pracować będziemy na jakiejś warstwie danych, przypominam, że dostęp do obecnie załadowanej do QGIS (dla ułatwienia) to:

In [ ]:
#obecnie zaznaczona warstwa
warstwa = iface.activeLayer()

Zmiany i modyfikacje nie są widoczne, dopóki nie odświeżymy widoku na warstwę. Możemy dokonać odświeżenia na warstwie lub w oknie mapy:

In [ ]:
#odświeżanie
warstwa.triggerRepaint() #metoda warstwy
#LUB
iface.mapCanvas().refresh() #metoda intefejsu, mapCanvas() zwraca referencję do "płótna" mapy

<h3> Dodawanie nowych obiektów </h3>

Zakładamy, że pracujemy na jakiejś warstwie wektorowej o charakterze LineString. Do wykładu dołączyłem taką warstwę.

Każdy obiekt (QgsFeature) warstwy wektorowej (QgsVectorLayer) ma:
<ul>
    <li> Atrybuty (QgsField) </li>
    <li> Geometrię (QgsGeometry) </li>
</ul>

Przy tworzeniu obiektów należy ustalić oba.

Najpierw tworzymy pusty obiekt:

In [ ]:
feat = QgsFeature(warstwa.fields()) #instancja obiektu QgsFeature, parametrem są pola wartości QgsField warstwy

Nie jest jeszcze przypisany do warstwy, zrobimy to później. Teraz nadajemy mu atrybuty oraz goemetrię:

In [ ]:
feat.setAttributes([0, 60]) #dodajemy atrybuty, ta warstwa ma dwa atrybuty integer
#LUB
feat.setAttribute(0,60) #tylko jeden wiersz

In [ ]:
#Tworzymy geometrię - linia z dwóch punktów
line_start = QgsPoint(15.74,50.89)
line_end = QgsPoint(15.73,50.91)

QgsGeometry to klasa geometrii, która może przyjmować postać punktów, linii i poligonów. Najczęściej budowana jest z punktów, czyli z klasy ([nowa klasa!]) QgsPoint:

    QgsPoint(x,y)

Następnie stosujemy jedno z poleceń .from __geometria__, czyli:
 
    fromPolyline, fromPolygonXY, fromPointXY lub fromWKT - czyli typu danych WKT

In [ ]:
feat.setGeometry(QgsGeometry.fromPolyline([line_start,line_end]))

I dodajemy go do warstwy. Do tego wykorzystujemy klasę dataProvider, która zawsze pośredniczy w operacjach dodawania, edytowania i usuwania obiektów warstw wektorowych. Więcej o niej poniżej.

In [ ]:
warstwa.dataProvider().addFeatures([feat]) #dodajemy obiekt do warstwy

Geometrie można budować z WKT, co jest o wiele prostsze, dlatego że w WKT można zapisać KAŻDY tym geometrii obecny w QGIS

<h3> WKT - Well-known text representation of geometry </h3>

To jeden ze standardów OGC, który pozwala na opisanie każdego typu geometrii:

    Point (jeden), MultiPoint (kolekcja),
    LineString (jeden), MultiLineString (kolekcja),
    Polygonjeden), MultiPolygon (kolekcja).

Definicja:

    Nazwa (x y)

Punkt:

    POINT (10 30) - Punkt o współrzędnych x=10 i y = 30

Linia:

    LINESTRING (30 10, 10 30) - to linia (prosta) o współrzędnych pierwszego wierzchołka X = 30, Y = 10 i drugiego X = 10, Y = 30

    LINESTRING (30 10, 10 30, 40 40) - to linia łamana, o trzech punktach
    
Poligon:

    POLYGON ((30 10, 40 40, 20 20, 10 20, 30 10))
    
    POLYGON ((35 10, 45 45, 15 40, 10 20, 35 10), (20 30, 35 35, 30 20, 20 30))
    
    Pierwsza i ostatnia współrzędna się powtarzają, bo wielobok musi mieć zamkniętą geometrię (czyli ma 4 wierzchołki)
    
    Zapis w podwójnym nawiasie umowżliwia podanie współrzędnych “otworu” w geometrii. Jak w drugim przykładzie.
    
    Kolekcje geometrii skłdają się z przynajmniej podwójnego zagnieżdżenia nawiasów. 
    
Naprawdę niezły przykład jest w tabelce na Wikipedii:

    https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry
    

<h3> WKB - Well-known binary </h3>

Binarny odpowiednik WKT. Zapis tych samych geometrii, ale językiem binarnym. Szybszy w składowaniu o dycztywywaniu.

Zadanie: Spróbuj dodać powyżej wprowadzoną linię za pomocę geometrii WKT. Geometrię zapisujemy jako tekst " ". Użyj polecenia. fromWKT()

<h2> Usuwanie obiektów </h2>

Wystarczy podać ich indeks:

In [ ]:
warstwa.dataProvider().deleteFeatures([0])

<h2> DataProvider </h2>

Ta klasa umożlwia dostęp do geometrii i atrybutów obiektów już obecnych na warstwie. Czyli jest przydatna przy dodawaniu, edycji i usuwaniu. Powyżej użyliśmy jej już do dodwania i usuwania. Użyjmy jej jeszcze do edycji!

<h2> Edycja obiektów </h2>

Dodajmy dwa obiekty na warstwie:

In [ ]:
warstwa = iface.activeLayer()

feat1 = QgsFeature(warstwa.fields())
feat1.setAttributes([0, 60])
line1_start = QgsPoint(15.74,50.89)
line1_end = QgsPoint(15.73,50.91)
feat1.setGeometry(QgsGeometry.fromPolyline([line1_start,line1_end]))

feat2 = QgsFeature(warstwa.fields())
feat2.setAttributes([0, 100])
line2_start = QgsPoint(11.74,53.89)
line2_end = QgsPoint(11.73,53.91)
feat2.setGeometry(QgsGeometry.fromPolyline([line2_start,line2_end]))
warstwa.dataProvider().addFeatures([feat1,feat2]) 

Teraz zmieńmy wartość podanych atrybutów pierwszego obiektu:

In [ ]:
attrs = { 0 : 100, 1 : 90 } #pole 1 otrzyma 100, pole 2 otrzyma 90
warstwa.dataProvider().changeAttributeValues({ 1 : attrs }) #obiekt o indeksie nr 1 zostanie zmieniony

Zmieńmy jeszcze geometrię:

In [ ]:
geom = QgsGeometry.fromPolyline([QgsPoint(15.70,50.59),QgsPoint(15.70,50.52),QgsPoint(15.73,50.52)])
warstwa.dataProvider().changeGeometryValues({1: geom})

<h3> Ostrożności nigdy za wiele - tryb edycji </h3>

Zmiany dokonane w trybie edycji mogą zostać odwrócone, dlatego też przed każdą decyzją najlepiej skorzystać jest z metod warstwy wektorowej:

    startEditing() - rozpocznij edycje
    commitChanges() - dokonaj zmian
    rollBack() - przywróć zmiany
    
    isEditable() - sprawdź czy warstwa jest edytowalna
    


<h3> Nowe atrybuty - jak dodać nowe atrybuty do warstwy</h3>

dataProvider() posiada metodę addAttributes oraz deleteAttributes.

[Nowa klasa] - QVariant, klasa przenosząca typ danych

In [ ]:
from qgis.PyQt.QtCore import QVariant #trzeba zaimportować, gdzyż ta warstwa przechowuje typy danych QGIS

warstwa.dataProvider().addAttributes\
([QgsField("nazwa_string", QVariant.String), QgsField("nazwa_int", QVariant.Int)])

In [ ]:
warstwa.dataProvider().deleteAttributes([0])

In [ ]:
warstwa.updateFields() #BARDZO WAŻNE, zawsze należy potwierdzić operację

<h3> Tworzenie warstw, zapisywanie </h3>

Zapisywanie warstw wektorowych obsługuje inna klasa - $QgsVectorFileWriter$ i jej funkcja $writeAsVectorFormat()$, która wspiera wszystkie formaty danych OGR (GeoPackage, Shapefile, GeoJSON, KML i inne)

In [ ]:
QgsVectorFileWriter.writeAsVectorFormat(warstwa, "nazwa", "UTF-8") #(warstwa, nazwa, kodowanie znaków)

In [ ]:
QgsVectorFileWriter.writeAsVectorFormat(warstwa, "nazwa", "UTF-8", driverName="GeoJSON")

In [ ]:
crs=QgsCoordinateReferenceSystem("epsg:4326")
QgsVecotFileWriter.writeAsVectorFormat(warswa, "nazwa", "UTF-8", crs = crs, driverName="GeoJSON")

<h3> Warstwy tymczasowe </h3>

W QGIS2 warstwy tymaczasowe nie były w ogóle dopuszczalne w aplikacji QGISa. W QGIS3 są, lecz nie zawsze. Należy pamiętać że nie zawsze nadają się do geoprocessingu. Pozwalają oszczędzić pamięć i miejsce na dysku twardym podczas operacji.

In [ ]:
from qgis.PyQt.QtCore import QVariant

#pusta warstwa w pamięci
temp_vlayer = QgsVectorLayer("Point", "tymczasowe_punkty", "memory") #zamiast "ogr" dajemy "memory", a zamiast nazwy jej charakter (punktowy "Point")
pr = temp_vlayer.dataProvider() #dataProvider warstwy tymaczasowej

#Atrybuty - pola
pr.addAttributes([QgsField("nazwa", QVariant.String), QgsField("wiek",QVariant.Int)])
temp_vlayer.updateFields()

#obiekt
feat = QgsFeature() #pusty obiekt
#Geometria
feat.setGeometry(QgsGeometry.fromPointXY(QgsPointXY(10,10)))
#Atrybuty
feat.setAttributes(["Bobby",12])
#Dodaj
pr.addFeatures([feat])
temp_vlayer.updateExtents() #odśwież warstwę

I teraz możemy ją dodac do okna mapy:

In [ ]:
QgsProject.instance().addMapLayer(temp_vlayer)

Tyle na dziś, dziękuję!